# Author Name Recon Part 2 - combine the author names with the affiliations
Following part 1, there are three main cases of ambiguity that can be resolved by affiliations and other methods, we do so here using a number of strategies. Given the disjointed nature of the affiliations data, I will also use this process to iteratively consolidate the two sets of affiliations for misspellings, missing data etc. present in the affiliations data. As well as checks for completeness of the affiliations data.

Pre-requisites:
- 020
- 021

Input files:
- processed author names from 020
- subset of combined affiliations from 021

Output files:
- combined processed author names with associated affiliations

In [276]:
import pandas as pd
# from unidecode import unidecode
import re
from datetime import date
import json
# import numpy as np
import string
import time
# from fuzzywuzzy import fuzz 
# from fuzzywuzzy import process 
# set column options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [277]:
base_path="/Users/sijiawu/Work/Thesis/Data/Affiliations/"

In [278]:
aff_sub=pd.read_pickle(base_path+"affiliations_combined_sub.pkl")

with open('author_proc.json') as f: 
    data = f.read() 

proc_auths_all = json.loads(data) 

j_data=pd.read_pickle(base_path.split('Aff')[0]+"Combined/020_merged_proc_scopus_inception_with_auth_split_2020.pkl")

In [279]:
# Are the ids unique? if true then we are a go!
temp=[]

for i in proc_auths_all.keys():
    temp.append(i.split("/")[-1])

len(temp)==len(list(set(temp)))

True

In [280]:
content_ex=['MISC',  'Discussion', 'Review', 'Review2',"Errata"]
content=['Article', 'Comment', 'Reply', 'Rejoinder']
jid=["aer","ecta","jpe","qje","res"]

In [281]:
proc_auths_all['https://doi.org/10.1093/qje/qjaa012']

{'authors': {'0': {'raw': 'Enke, Benjamin',
   'init': 'benjamin enke',
   'auth_suffix': [],
   'a1': 'benjamin enke',
   'a2': 'b. enke',
   'a3': 'b. enke'}},
 'year': 2020,
 'content_type': 'Article',
 'jid': 'qje'}

In [282]:
key_match={}
for i in proc_auths_all.keys():
    auths=proc_auths_all[i]["authors"].keys()
    key_match[i.split('/')[-1]]=i
    for j in auths:
        proc_auths_all[i]['authors'][j]['affs']={}

In [283]:
aff_sub['auth_num']=aff_sub['auth_num'].astype(int)

In [284]:
g=[]
issues=[]
for i in aff_sub.index:
    g.append(key_match[str(aff_sub.loc[i,'id'])])
    # print(key_match[str(aff_sub.loc[i,'id'])])
    # print(proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['authors'])
    # print(str(aff_sub.loc[i,'auth_num']))
    if proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['content_type'] in content_ex:
        continue        
    if proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['year']<1940:
        continue
    try:
        auth_ob=proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['authors'][str(aff_sub.loc[i,'auth_num'])]['affs']
        proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['authors'][str(aff_sub.loc[i,'auth_num'])]["alt_auth"]=aff_sub.loc[i,"author"]
        auth_ob[aff_sub.loc[i, 'aff_num']]=aff_sub.loc[i,['aff_main_final', 'aff_subunit_final','aff_department_final', 'alt_final', 'manual', 'screened']].to_dict()
    except:
        print(proc_auths_all[key_match[str(aff_sub.loc[i,'id'])]]['authors'])
        print(str(aff_sub.loc[i,'auth_num']))
        issues.append(key_match[str(aff_sub.loc[i,'id'])])
    

In [285]:
len(issues)

0

In [286]:
import pprint

In [287]:
missing_data_affs={"https://doi.org/10.1093/qje/qjaa012":{"0":{1: {
    'aff_department_final': None,
    'aff_main_final': 'harvard university',
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
    },
2:{
    'aff_department_final': None,
    'aff_main_final':"national bureau of economic research - nber",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
}
}},
'https://www.jstor.org/stable/26921614': {
      '0': {1:
            {
    'aff_department_final': None,
    'aff_main_final': "boston university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }
},
      '1': {1:
            {
    'aff_department_final': None,
    'aff_main_final': "shandong university of finance and economics",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            },
            2:
            {
    'aff_department_final': None,
    'aff_main_final': "macquarie university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }},
      '2': {1:
            {
    'aff_department_final': None,
    'aff_main_final': "university of southern california",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            },2:
            {
    'aff_department_final': None,
    'aff_main_final': "shanghai jiaotong university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }},
      '3': {1:
            {
    'aff_department_final': None,
    'aff_main_final': "university of international business and economics",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }}
      },
    "https://www.jstor.org/stable/43821401":{
        "0":{1:
            {
    'aff_department_final': None,
    'aff_main_final': "mit",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            },
            2:
            {
    'aff_department_final': None,
    'aff_main_final': "national bureau of economics - nber",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }},
        "1":{1:
            {
    'aff_department_final': None,
    'aff_main_final': "bocconi university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            },
            2:
            {
    'aff_department_final': None,
    'aff_main_final': "IGIER",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }},
        "2":{1:
            {
    'aff_department_final': None,
    'aff_main_final': "columbia university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }}
    },
    "https://www.jstor.org/stable/42920997":{"0":{1:
            {
    'aff_department_final': None,
    'aff_main_final': "stanford university",
    'aff_subunit_final': None,
    'alt_final': None,
    'manual': 'manual',
    'screened': 'NA'
            }}}  
      
}

In [288]:
for i in missing_data_affs.keys():
    print(i)
    print(proc_auths_all[i])
    for j in proc_auths_all[i]['authors'].keys():
        proc_auths_all[i]['authors'][j]["affs"]=missing_data_affs[i][j]


https://doi.org/10.1093/qje/qjaa012
{'authors': {'0': {'raw': 'Enke, Benjamin', 'init': 'benjamin enke', 'auth_suffix': [], 'a1': 'benjamin enke', 'a2': 'b. enke', 'a3': 'b. enke', 'affs': {}}}, 'year': 2020, 'content_type': 'Article', 'jid': 'qje'}
https://www.jstor.org/stable/26921614
{'authors': {'0': {'raw': 'Raymond Fisman', 'init': 'raymond fisman', 'auth_suffix': [], 'a1': 'raymond fisman', 'a2': 'r. fisman', 'a3': 'r. fisman', 'affs': {}}, '1': {'raw': 'Jing Shi', 'init': 'jing shi', 'auth_suffix': [], 'a1': 'jing shi', 'a2': 'j. shi', 'a3': 'j. shi', 'affs': {}}, '2': {'raw': 'Yongxiang Wang', 'init': 'yongxiang wang', 'auth_suffix': [], 'a1': 'yongxiang wang', 'a2': 'y. wang', 'a3': 'y. wang', 'affs': {}}, '3': {'raw': 'Weixing Wu', 'init': 'weixing wu', 'auth_suffix': [], 'a1': 'weixing wu', 'a2': 'w. wu', 'a3': 'w. wu', 'affs': {}}}, 'year': 2020, 'content_type': 'Article', 'jid': 'aer'}
https://www.jstor.org/stable/43821401
{'authors': {'0': {'raw': 'George-Marios Angeleto

In [289]:
with open(base_path+"author_proc_w_aff.json", "w") as outfile: 
    json.dump(proc_auths_all, outfile, indent=4, default=int)

## Checks on the affiliations data

In [290]:
f=[]
n=[]
m=[]
l=[]
t=[]
c_type=[]
c_type_c=[]
nones=[]
missing=[]
other=[]
no_auths=[]
for i in proc_auths_all.keys():
    if (proc_auths_all[i]["content_type"] in content) & (proc_auths_all[i]["year"]>=1940):
        if len(proc_auths_all[i]['authors'])==0:
            no_auths.append(i)
            continue
        if (len(proc_auths_all[i]["authors"]['0']['affs'])!=0):
            f.append(i)
            # print(i)
        else:
            print(i)
            pprint.pprint(proc_auths_all[i])
            c_type.append(str(proc_auths_all[i]["year"])+" "+proc_auths_all[i]["content_type"])
            c_type_c.append(proc_auths_all[i]["content_type"])
            missing.append(i)
    else:
        m.append(i)

https://www.jstor.org/stable/23469808
{'authors': {'0': {'a1': 'wendy a. stock',
                   'a2': 'w. a. stock',
                   'a3': 'w. stock',
                   'affs': {},
                   'auth_suffix': [],
                   'init': 'wendy a. stock',
                   'raw': 'Wendy A. Stock'},
             '1': {'a1': 'john j. siegfried',
                   'a2': 'j. j. siegfried',
                   'a3': 'j. siegfried',
                   'affs': {},
                   'auth_suffix': [],
                   'init': 'john j. siegfried',
                   'raw': 'John J. Siegfried'}},
 'content_type': 'Article',
 'jid': 'aer',
 'year': 2013}
https://www.jstor.org/stable/41724643
{'authors': {'0': {'a1': 'michael faye',
                   'a2': 'm. faye',
                   'a3': 'm. faye',
                   'affs': {},
                   'auth_suffix': [],
                   'init': 'michael faye',
                   'raw': 'Michael Faye'},
             '1': {'a1

In [291]:
len(missing)

133

In [292]:
# df=[]
# k=0
# aString="https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/"
# for i in missing:
#     k+=1
#     x=proc_auths_all[i]
#     authstring=[]
#     for j in x['authors'].keys():
#         authstring.append(x['authors'][j]['raw'])
#     results = j_data.loc[j_data["URL"] == i]

#     au={
#         "pdf_url": aString+i.split('/')[-1]+"_wo_cover.pdf",
#         "authors": ";".join(authstring),
#         "journal":list(results["journal"])[0],
#         "pages": list(results["pages"])[0],
#         'title':list(results["title"])[0],
#         'vol':list(results["volume"])[0],
#         'issue':list(results["number"])[0],
#         'id': i.split('/')[-1],
#         "year": x['year']
#     }
    
#     df.append(au)

# df2=[]
# k=0
# aString="https://myawsbucket-1231.s3.eu-west-3.amazonaws.com/missing/"
# for i in other:
#     k+=1
#     x=proc_auths_all[i]
#     authstring=[]
#     for j in x['authors'].keys():
#         authstring.append(x['authors'][j]['raw'])
#     results = j_data.loc[j_data["URL"] == i]

#     au={
#         "pdf_url": aString+i.split('/')[-1]+"_wo_cover.pdf",
#         "authors": ";".join(authstring),
#         "journal":list(results["journal"])[0],
#         "pages": list(results["pages"])[0],
#         'title':list(results["title"])[0],
#         'vol':list(results["volume"])[0],
#         'issue':list(results["number"])[0],
#         'id': i.split('/')[-1],
#         "year": x['year']
#     }
    
#     df2.append(au)


In [293]:
# df=pd.DataFrame(df)
# df2=pd.DataFrame(df2)

# df.to_csv( "Remnant_formatted.csv",encoding="UTF-8", quoting=1, index=False)



In [294]:

# jnames={
#         'The Quarterly Journal of Economics':"/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/QJE/qje_wo_cover/",
#         'The Review of Economic Studies':"/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/RES/res_wo_cover/", 
#         'Econometrica': "/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/ECTA/ecta_wo_cover/",
#         'Journal of Political Economy':"/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/JPE/jpe_wo_cover/", 
#         'The American Economic Review':"/Users/sijiawu/Dropbox/80YearsEconomicResearch/Data/0_PDF/AER/aer_wo_cover/"
#        }

# import glob
# import os.path
# import shutil
# from os.path import exists
# for i in df.index:
    
#     fp=jnames[df.loc[i,"journal"]]+df.loc[i,"id"]+"_wo_cover.pdf"
#     file_exists = exists(fp)
#     if file_exists:
#         TARG_DIR = "/Users/sijiawu/Dropbox/80YearsEconomicResearch/Missing"
#         shutil.copy(fp,TARG_DIR)


# df2.head()
# df2[["journal","year"]].value_counts()